In [ ]:
!pip install datasets transformers sentencepiece
!pip install huggingface_hub
!pip install accelerate -U
!pip install transformers[torch] -U

In [ ]:
import os
!pip install huggingface_hub

!apt install git-lfs
!git lfs install

from huggingface_hub import notebook_login
notebook_login()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Git LFS initialized.


In [ ]:
import os
import json
import torch
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import RobertaTokenizerFast, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset

from tqdm.notebook import tqdm
from IPython.display import HTML
from functools import partial
from difflib import SequenceMatcher

pd.set_option("display.max_columns", None)

dataset_directory = '/content/drive/MyDrive/Codehealer/'

os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
Codehealer = load_dataset("json", data_files={"train": dataset_directory+"codenetpy_train.json", "test": dataset_directory+"codenetry_test.json"}, field='data')

filtered_train_dataset = Codehealer["train"].filter(lambda code_sample: code_sample["returncode"] != 0).train_test_split(test_size=0.1)
filtered_test_dataset = Codehealer["test"]

FileNotFoundError: Unable to find '/content/drive/MyDrive/Codehealer/codenetpy_train.json'

In [ ]:
import torch
from transformers import RobertaTokenizerFast
from functools import partial

max_input_length = 256
max_output_length = 512

def process_and_tokenize_samples(tokenizer, code_sample):
    tokenized_inputs = tokenizer(text=code_sample["error_classification"], text_pair=code_sample["original_src"], max_length=max_input_length, padding=True, truncation=True)
    tokenized_targets = tokenizer(code_sample["changed_src"], max_length=max_output_length, padding=True, truncation=True)

    label_ids = tokenized_targets.input_ids
    label_ids = torch.tensor(label_ids)
    label_ids[label_ids == tokenizer.pad_token_id] = -100

    tokenized_inputs["labels"] = label_ids.tolist()
    return tokenized_inputs

tokenizer_instance = RobertaTokenizerFast.from_pretrained("Salesforce/codet5-base")

tokenized_train_dataset = filtered_train_dataset.map(partial(process_and_tokenize_samples, tokenizer_instance), batched=True, batch_size=4, remove_columns=filtered_train_dataset["train"].column_names)
tokenized_test_dataset = filtered_test_dataset.map(partial(process_and_tokenize_samples, tokenizer_instance), batched=True, batch_size=4, remove_columns=filtered_test_dataset.column_names)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

NameError: name 'filtered_train_dataset' is not defined

In [ ]:
from transformers import T5ForConditionalGeneration, TrainingArguments, DataCollatorForSeq2Seq
import os

output_directory_path = '/content/codet5-base-bug-fix-outputs'
logging_directory_path = '/content/codet5-base-logs'

os.makedirs(output_directory_path, exist_ok=True)
os.makedirs(logging_directory_path, exist_ok=True)

training_args = TrainingArguments(
    output_dir=output_directory_path,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=logging_directory_path,
    logging_steps=1_000,
    save_strategy="no",
    push_to_hub=False,
)

model_instance = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-base")

data_collator_instance = DataCollatorForSeq2Seq(tokenizer_instance, model_instance, padding=True)

In [ ]:
def evaluate_model(predictions_and_labels):
    predictions, labels = predictions_and_labels

    predictions = np.argmax(predictions[0], axis=2)

    flattened_predictions = [pred for sequence in predictions for pred in sequence]
    flattened_labels = [label for sequence in labels for label in sequence]

    return {
        "precision": precision_score(flattened_labels, flattened_predictions, average="weighted"),
        "recall": recall_score(flattened_labels, flattened_predictions, average="weighted"),
        "f1": f1_score(flattened_labels, flattened_predictions, average="weighted"),
        "accuracy": accuracy_score(flattened_labels, flattened_predictions)
    }

In [ ]:
training_instance = Trainer(
    model=model_instance,
    args=training_args,
    train_dataset=tokenized_train_dataset["train"],
    eval_dataset=tokenized_train_dataset["test"].select(range(10)),
    tokenizer=tokenizer_instance,
    compute_metrics=evaluate_model,
    data_collator=data_collator_instance,
)

training_instance.train()

In [ ]:
training_instance.evaluate()

In [ ]:
saved_model_directory_path = '/content/drive/MyDrive/saved_model_epoch'

os.makedirs(saved_model_directory_path, exist_ok=True)

In [ ]:
model_instance.save_pretrained(saved_model_directory_path)

In [ ]:
model_instance = T5ForConditionalGeneration.from_pretrained(saved_model_directory_path)

In [ ]:
def generate_code_suggestions(tokenizer_instance, model_instance, error_message, original_code_snippet, beam_search_size=5):
    tokenized_inputs = tokenizer_instance(text=error_message, text_pair=original_code_snippet, max_length=512, padding=True, truncation=True, return_tensors="pt").to(model_instance.device)
    generated_code_ids = model_instance.generate(num_beams=beam_search_size, no_repeat_ngram_size=2, num_return_sequences=beam_search_size, max_length=512, **tokenized_inputs).cpu().detach().numpy()

    return tokenizer_instance.batch_decode(generated_code_ids, skip_special_tokens=True)

In [ ]:
error_message_example = "SyntaxError: Missing parentheses in call to 'print'. Did you mean print(l[i])?"
code_snippet_example = "l=[]\nwhile True:\n    try:\n        l.append(input())\n    except SyntaxError:\n        break\nl.sort(reverse = True)\nfor i in range(3):\n    print l[i]"

tokenized_inputs_example = tokenizer_instance(text=error_message_example, text_pair=code_snippet_example, max_length=512, padding=True, truncation=True)

generated_code_example = generate_code_suggestions(tokenizer_instance, model_instance, [error_message_example], [code_snippet_example], beam_search_size=5)[0]

print("Original Code:")
print(code_snippet_example)
print("Suggested Fix:")
print(generated_code_example)
